In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback
from peft import LoraConfig, TaskType, get_peft_model
import torch
import psutil
import numpy as np



2025-06-08 17:25:37.018149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749403537.202680      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749403537.257952      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
torch.cuda.set_per_process_memory_fraction(1.0, 0)  # Use maximum available memory
torch.cuda.memory_max_split_size_mb = 64  # Set the max split size to avoid fragmentation

In [3]:
def print_memory_footprint():
    # GPU memory usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        gpu_memory_cached = torch.cuda.memory_reserved() / (1024 ** 3)  # Cached memory
        print(f"[GPU] Memory Allocated: {gpu_memory:.2f} GB, Cached: {gpu_memory_cached:.2f} GB")
    else:
        print("[GPU] No GPU detected.")

    # CPU memory usage
    memory = psutil.virtual_memory()
    used_memory_gb = memory.used / (1024 ** 3)  # Convert to GB
    total_memory_gb = memory.total / (1024 ** 3)
    print(f"[CPU] Memory Usage: {used_memory_gb:.2f} GB / {total_memory_gb:.2f} GB")

In [4]:
'''
print("First example of blended_skill_talk training set:")
print(dataset['train'][0])
'''

'\nprint("First example of blended_skill_talk training set:")\nprint(dataset[\'train\'][0])\n'

In [5]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token, so we use eos_token

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [6]:
# # Load dataset and tokenizer
# dataset2 = load_dataset("daily_dialog")



# def tokenize_function(examples):
#     # Concatenate dialog turns into a single string for language modeling
#     texts = [" ".join(dialog) for dialog in examples["dialog"]]
#     return tokenizer(texts, truncation=True, max_length=512)

# # Tokenize datasets
# tokenized_datasets = dataset2.map(tokenize_function, batched=True, remove_columns=["dialog", "act", "emotion"])
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
# small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

In [7]:

# print(dataset2)
# tokenized_datasets

In [8]:
dataset = load_dataset("knkarthick/dialogsum")


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [9]:
def tokenize_function(examples):
    # Concatenate dialog turns into a single string for language modeling
    texts = [" ".join(dialog) for dialog in examples["dialogue"]]
    return tokenizer(texts, truncation=True, max_length=512)


# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['id', 'dialogue', 'summary', 'topic'])
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [10]:
print(dataset)
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1500
    })
})

In [11]:
dataset["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 12460
})

In [12]:
# LoRA configuration for causal language modeling
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=2,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj"],
)

# %%

In [13]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained GPT-2 language model
# Load pre-trained GPT-2 model
#model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                             torch_dtype=torch.bfloat16,).to(device)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Enable gradient checkpointing if you run into memory issues
#model.gradient_checkpointing_enable()

# %%
# Print the model's architecture to inspect the names of the modules
#print(model)

# %%
print_memory_footprint()

Using device: cuda


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 1.80 GB / 31.35 GB


In [14]:
# %%

# Data collator for language modeling (masks tokens for prediction)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # mlm=False for causal LM

# Perplexity as metric
#perplexity_metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    shift_logits = torch.tensor(logits[:, :-1, :])
    shift_labels = torch.tensor(labels[:, 1:])
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    ppl = torch.exp(loss).item()
    return {"perplexity": ppl}

# Training args
training_args = TrainingArguments(
    output_dir="./lama-dialog-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=1000,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    bf16=True,  # Enable bfloat16
    fp16=False,  # Disable fp16 to avoid conflicts
)
# %%
print_memory_footprint()

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 1.81 GB / 31.35 GB


In [15]:
class MemoryCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        print("\nMemory footprint after evaluation:")
        print_memory_footprint()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    #eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()]
)


/tmp/ipykernel_35/3187942852.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:

print("--- TinyLlama Before Fine-tuning ---")
model_before_finetune = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
             torch_dtype=torch.bfloat16).to(device)

input_text_before = "Hello, how are you?"
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input: Hello, how are you?
Generated Response (Before): Hello, how are you?
I am fine, thank you.
How are you? I am fine, thank you.
Can you repeat that again?
Can you repeat that again, please?
Can you repeat that again,
---


In [17]:

print("--- TinyLlama Before Fine-tuning ---")
#model_before_finetune = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

input_text_before = "The USA celebrate independence day on "
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---
Input: The USA celebrate independence day on 
Generated Response (Before): The USA celebrate independence day on 4th July.

2. The USA celebrate independence day on 4th July.

3. The USA celebrate independence day on 4th July.

4. The
---


In [18]:
any(p.requires_grad for p in model.parameters())

True

In [19]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=500, training_loss=0.5579511108398437, metrics={'train_runtime': 1303.9512, 'train_samples_per_second': 1.534, 'train_steps_per_second': 0.383, 'total_flos': 6355636593573888.0, 'train_loss': 0.5579511108398437, 'epoch': 1.0})

In [20]:
trainer.save_model('TinyLlama-new-1000')

In [21]:
print_memory_footprint()

[GPU] Memory Allocated: 4.12 GB, Cached: 8.11 GB
[CPU] Memory Usage: 2.63 GB / 31.35 GB


In [30]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For generation, we'll load the model and use the `generate` method
model_for_generation = AutoModelForCausalLM.from_pretrained('TinyLlama-new-1000').to(device)
#model_for_generation = get_peft_model(model_for_generation, lora_config) # Ensure LoRA is applied

input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")

print_memory_footprint()

Using device: cuda

Generating response:
Input: Hello, how are you?
Generated Response: Hello, how are you?

Jane:
(smiling)
I'm doing well, thank you.

Tom:
(smiling back)
Me too.

Jane:
(smiling)
I'm glad to hear that.

Tom:
(smiling)
Me too.

Jane:
(smiling)
I'm glad we're finally meeting.

Tom:
(smiling)
Me too.

Jane:
(smiling)
So, what brings you to our city?

Tom:
(smiling)
I'm here to visit my sister.

Jane
[GPU] Memory Allocated: 8.22 GB, Cached: 12.88 GB
[CPU] Memory Usage: 6.28 GB / 31.35 GB


In [23]:
input_text = "The USA celebrate independence day on "
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: The USA celebrate independence day on 
Generated Response: The USA celebrate independence day on 4th July.

4. The USA celebrate independence day on 4th July.

5. The USA celebrate independence day on 4th July.

6. The USA celebrate independence day on 4th July.

7. The USA celebrate independence day on 4th July.

8. The USA celebrate independence day on 4th July.

9. The USA celebrate independence day on 4th July.

10. The USA celebrate independence day on 4th July.

11. The USA celebrate independence day on 4th July.

12. The USA


In [27]:
input_text = "Explain how gravity works. Can you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=250,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: Explain how gravity works. Can you?
Generated Response: Explain how gravity works. Can you?
